In [1]:
import yfinance as yf
import talib
import pandas as pd
import numpy as np
import vectorbt as vbt 
from vectorbt.portfolio.enums import OrderSide

# data preprocess
target_stock = "00632R"
stock_df = pd.read_csv(r'C:\Users\Admin\Desktop\量化交易\lstm 預測股價(squeeze中包含的技術分析)\LSTM_stock_predicting\data\00632R 2023.csv', encoding='utf-8', encoding_errors= 'ignore')
five_day_min_low = talib.MIN(stock_df['Low'], timeperiod=5) # 五天最低價當作long strategy的停損點
five_day_max_high = talib.MAX(stock_df['High'], timeperiod=5) # 五天最高價當作short strategy的停損點
stock_df['five_day_min_low'] = five_day_min_low
stock_df['five_day_max_high'] = five_day_max_high
stock_df = stock_df[25:-1]
predicted_price_df = pd.read_csv(r"C:\Users\Admin\Desktop\量化交易\lstm 預測股價(squeeze中包含的技術分析)\LSTM_stock_predicting\predicted_result\GRUModel2 00632R 2023 predicted prices.csv", encoding='utf-8', encoding_errors = 'ignore')
stock_df['Date'] = pd.to_datetime(stock_df['Date'])
stock_df.index = stock_df['Date']
stock_df['predicted_price'] = predicted_price_df['predicted_price'].values
stock_df['true_price'] = predicted_price_df['true_price'].values

# long short condition
stock_df['predicted_price_up'] = (stock_df['predicted_price'] > stock_df['predicted_price'].shift(1)).astype(int)
stock_df['actual_price_up'] = (stock_df['Close'] > stock_df['Close'].shift(1)).astype(int)
stock_df['long_condition'] = stock_df['true_price'].shift(1)<=stock_df['predicted_price'] 
stock_df['short_condition'] = stock_df['true_price'].shift(1)>stock_df['predicted_price'] 
stock_df['long_condition'] = stock_df['long_condition'].astype(int)
stock_df['short_condition'] = stock_df['short_condition'].astype(int)

stock_df

c:\Users\Admin\anaconda3\envs\finance\lib\site-packages\numba\core\decorators.py:262: NumbaDeprecationWarning: numba.generated_jit is deprecated. Please see the documentation at: https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-generated-jit for more information and advice on a suitable replacement.
  warnings.warn(msg, NumbaDeprecationWarning)


,Date,Capacity,Turnover,Open,High,Low,Close,Change,Transcation,buy_Dealer_Hedging,...,sell_Foreign_Investor,sell_Investment_Trust,five_day_min_low,five_day_max_high,predicted_price,true_price,predicted_price_up,actual_price_up,long_condition,short_condition
Date,,,,,,,,,,,,,,,,,,,,,
2023-02-17,2023-02-17,138377640,745426970,5.38,5.40,5.37,5.39,0.05,5567,57635130,...,30164000,0,5.30,5.41,0.023888,0.023591,0,0,0,0
2023-02-20,2023-02-20,89541534,480474496,5.39,5.40,5.34,5.36,-0.03,4358,3025000,...,721000,0,5.30,5.41,0.023996,0.021625,1,0,1,0
2023-02-21,2023-02-21,64450588,345522161,5.37,5.38,5.35,5.35,-0.01,3257,11929000,...,1280000,0,5.33,5.41,0.023972,0.020970,0,0,1,0
2023-02-22,2023-02-22,198489210,1075006812,5.41,5.43,5.40,5.41,0.06,8735,15898000,...,537000,0,5.33,5.43,0.024592,0.024902,1,1,1,0
2023-02-23,2023-02-23,227296728,1214002585,5.37,5.38,5.31,5.33,-0.08,12136,15745000,...,748000,0,5.31,5.43,0.024955,0.019659,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-12-22,2023-12-22,50369892,230149791,4.56,4.58,4.56,4.57,0.00,1518,8953000,...,3098000,0,4.53,4.60,-0.028594,-0.030144,0,0,1,0
2023-12-25,2023-12-25,40493836,184314943,4.55,4.56,4.54,4.56,-0.01,1402,11014000,...,0,0,4.53,4.60,-0.028495,-0.030799,1,0,1,0
2023-12-26,2023-12-26,90193621,408919172,4.55,4.55,4.52,4.53,-0.03,3443,9213000,...,0,0,4.52,4.60,-0.028493,-0.032765,1,0,1,0


In [2]:
# long strategy
signals_long = np.zeros(len(stock_df))
partition_size_long = 0
stop_loss_long = 0
for i in range(len(stock_df)):
    if partition_size_long == 0:
        if stock_df['long_condition'][i] == 1:
            signals_long[i] = 1
            partition_size_long = 1
            stop_loss_long = stock_df['five_day_min_low'][i]
    if partition_size_long == 1:
        if (stock_df['Close'][i] <= stop_loss_long) or (stock_df['short_condition'][i] == 1): # 停利停損
            signals_long[i] = -1
            stop_loss_long = 0 # 停損點重置
            partition_size_long = 0 # 平倉

# short strategy
signals_short = np.zeros(len(stock_df))
partition_size_short = 0
stop_loss_short = 0
for i in range(len(stock_df)):
    if partition_size_short == 0:
        if stock_df['short_condition'][i] == 1:
            signals_short[i] = -1
            partition_size_short = -1
            stop_loss_short = stock_df['five_day_max_high'][i]
    if partition_size_short == -1:
        if (stock_df['Close'][i] >= stop_loss_short) or (stock_df['long_condition'][i] == 1):# 停利停損
            signals_short[i] = 1
            stop_loss_short = 0 # 停損點重置
            partition_size_short = 0 # 平倉
            
entries_long = signals_long == 1
exits_long = signals_long == -1
entries_short = signals_short == -1
exits_short = signals_short == 1
pf = vbt.Portfolio.from_signals(stock_df['Open'], 
                                    entries=entries_long,
                                    exits=exits_long, 
                                    short_entries=entries_short,
                                    short_exits=exits_short,
                                    fees = 0,
                                    freq='1D',
                                    direction='both',)
# print(pf_long.stats().to_string()) # to_string()可以將全部結果攤開
print(pf.stats().to_string())
'''entries_short = signals_short == -1
exits_short = signals_short == 1
pf_short = vbt.Portfolio.from_signals(data['Close'], entries_short, exits_short, direction='shortonly')'''
# print(pf_short.stats().to_string())

C:\Users\Admin\AppData\Local\Temp\ipykernel_39072\419594594.py:7: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if stock_df['long_condition'][i] == 1:
C:\Users\Admin\AppData\Local\Temp\ipykernel_39072\419594594.py:10: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  stop_loss_long = stock_df['five_day_min_low'][i]
C:\Users\Admin\AppData\Local\Temp\ipykernel_39072\419594594.py:12: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if (stock_

Start                               2023-02-17 00:00:00
End                                 2023-12-28 00:00:00
Period                                213 days 00:00:00
Start Value                                       100.0
End Value                                    120.158125
Total Return [%]                              20.158125
Benchmark Return [%]                         -16.542751
Max Gross Exposure [%]                            100.0
Total Fees Paid                                     0.0
Max Drawdown [%]                               6.335949
Max Drawdown Duration                  39 days 00:00:00
Total Trades                                         64
Total Closed Trades                                  63
Total Open Trades                                     1
Open Trade PnL                                      0.0
Win Rate [%]                                  49.206349
Best Trade [%]                                 3.333333
Worst Trade [%]                               -2

"entries_short = signals_short == -1\nexits_short = signals_short == 1\npf_short = vbt.Portfolio.from_signals(data['Close'], entries_short, exits_short, direction='shortonly')"

In [3]:
pf.plot().show()